In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_pickle('data.pkl')
data = data[data.date_block_num > 11]

In [3]:
drop_features = ['date_cat-medium-type_avg_item_cnt_lag_1', 'date_city_avg_item_cnt_lag_2', 'date_shop_avg_item_cnt_lag_6', 'days_in_month', 'delta_price_trend_with_lag_1', 'delta_price_trend_with_lag_4', 'delta_shop_revenue_this_month', 'delta_shop_revenue_this_month_lag_2', 'item_avg_price_during_month_lag_3']
data.drop(drop_features, axis=1, inplace=True)

In [4]:
X = None
Y = None
x = None

def set_X_Y_x():
    global X, Y, x
    X = data[data.date_block_num < 34].drop(['item_cnt_month'], axis=1)
    Y = data[data.date_block_num < 34]['item_cnt_month']
    x = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

set_X_Y_x()

In [5]:
from sklearn.metrics import mean_squared_error

def rmse(y_te, y_p):
    return np.sqrt(mean_squared_error(y_te, y_p))

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [7]:
# X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.65, random_state=42)
# lf = LinearRegression()
# lf.fit(X_train, y_train)
# prediction = lf.predict(X_test).clip(0,20)
#
# print(f"RMSE: {rmse(y_test, prediction)}")

In [8]:
# data = data[data.item_price<100000]
# data = data[data.item_cnt_month<1001]
# set_X_Y_x()

In [9]:
# X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.65, random_state=42)
# lf = LinearRegression()
# lf.fit(X_train, y_train)
# prediction = lf.predict(X_test).clip(0,20)
#
# print(f"RMSE: {rmse(y_test, prediction)}")

In [10]:
# test_data = pd.read_csv('data/technical/test.csv')
# shops_in_test_data = set()
#
# for i, row in test_data.iterrows():
#     shops_in_test_data.add(row['shop_id'])

In [11]:
# all_shops = set([*range(0, 60)])
# shops_not_present = all_shops-shops_in_test_data
# shops_not_present

In [12]:
# data.shape

In [13]:
# for missing_shop in shops_not_present:
#     index_names = data[ data['shop_id'] == missing_shop ].index
#     data.drop(index_names, inplace=True)

In [14]:
# data.shape

In [15]:
# X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.65, random_state=42)
# lf = LinearRegression()
# lf.fit(X_train, y_train)
# prediction = lf.predict(X_test).clip(0,20)
#
# print(f"RMSE: {rmse(y_test, prediction)}")


In [16]:
# from sklearn.model_selection import cross_val_score
#
# X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.65, random_state=42)
#
# clf = LinearRegression()
# score = cross_val_score(clf, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
# print('RMSE %2f' %(-1 * score.mean()))

In [17]:
from sklearn.linear_model import Ridge


X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.65, random_state=42)

model = Ridge()
model.fit(X_train, y_train)

prediction = model.predict(X_test).clip(0,20)
print(f"RMSE: {rmse(y_test, prediction)}")

/home/ced/Apps/miniconda3/envs/rp/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=1.13384e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


RMSE: 0.8003265204478858


In [18]:
# from sklearn.linear_model import Lasso
# from sklearn.model_selection import GridSearchCV
#
#
# X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.65, random_state=42)
#
# alphas = np.logspace(-3, 1, 10)
# parameter_candidates = [{'alpha': alphas, 'fit_intercept': [True, False]}]
#
# model = Lasso()
#
# clf = GridSearchCV(estimator=model, param_grid=parameter_candidates, n_jobs=-1)
# clf.fit(X_train, y_train)
#
# print(f'Best estimators: {clf.best_estimator_}')
# clf = clf.best_estimator_
#
# prediction = clf.predict(X_test)
# print(f"RMSE: {rmse(y_test, prediction)}")
#